In [8]:
from impy import models as im
from impy.kinematics import CenterOfMass
from impy.constants import GeV
from impy.util import get_all_models
from util import run_in_separate_process
from particle import literals as lp
import numpy as np
import joblib

In [9]:
get_all_models()

[impy.models.dpmjetIII.DpmjetIII191,
 impy.models.dpmjetIII.DpmjetIII193,
 impy.models.dpmjetIII.DpmjetIII306,
 impy.models.epos.EposLHC,
 impy.models.phojet.Phojet112,
 impy.models.phojet.Phojet191,
 impy.models.phojet.Phojet193,
 impy.models.pythia6.Pythia6,
 impy.models.pythia8.Pythia8,
 impy.models.qgsjet.QGSJet01d,
 impy.models.qgsjet.QGSJetII03,
 impy.models.qgsjet.QGSJetII04,
 impy.models.sibyll.Sibyll21,
 impy.models.sibyll.Sibyll23,
 impy.models.sibyll.Sibyll23c,
 impy.models.sibyll.Sibyll23d,
 impy.models.sophia.Sophia20,
 impy.models.urqmd.UrQMD34]

In [ ]:
Model(CenterOfMass(energies[0], "p", "p"))

In [6]:
models = [im.dpmjetIII.DpmjetIII191,
 im.dpmjetIII.DpmjetIII193,
 im.dpmjetIII.DpmjetIII306,
 im.epos.EposLHC,
 im.phojet.Phojet112,
 im.phojet.Phojet191,
 im.phojet.Phojet193]

In [7]:
energies = np.geomspace(10, 1000, 20) * GeV

@joblib.delayed
def run(Model):
    values = []
    m = Model(CenterOfMass(energies[0], "p", "p"))
    for en in energies:
        kin = CenterOfMass(en, "p", "p")
        c = m.cross_section(kin)
        values.append(c)
    return values

Models = [M for M in models if lp.proton.pdgid in M.projectiles]
cross_sections = {}
out = joblib.Parallel(len(Models), batch_size=1)(run(Model) for Model in Models)
for Model, o in zip(Models, out):
    cross_sections[Model.pyname] = o

 +-----------------------------------------------------------------------+
 |                                                                       |
 |                       DPMJET-III version 19.3.3                       |
 |                                                                       |
 |                      (Last change:  30 Aug 2022)                      |
 |                                                                       |
 |            Authors:                                                   |
 |                     Stefan Roesler     (CERN)                         |
 |                     Anatoli Fedynitch (ASIoP)                         |
 |                     Ralph Engel        (KIT)                          |
 |                     Johannes Ranft     (Siegen Univ.)                 |
 |                                                                       |
 |        https://github.com/afedynitch/dpmjet                           |
 |                       

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGFPE(-8)}

In [6]:
from matplotlib import pyplot as plt
import dataclasses

for model in cross_sections:
    if model == "Pythia8":
        continue
    plt.figure()
    kinds = "total", "inelastic", "diffractive"
    for k, model2 in enumerate(("Pythia8", model)):
        data = cross_sections[model2]
        for i, kind in enumerate(kinds):
            if kind == "diffractive":
                val = [sum(
                    (0 if np.isnan(v) else v) for k, v in dataclasses.asdict(d).items() if "diffractive" in k) for d in data]
            else:
                val = [getattr(d, kind) for d in data]

            plt.plot(energies / GeV, val, label=model2 + " " + kind,
                    ls=["-", "--", ":"][i], color=["k", "C1"][k], zorder=1 + k)
    plt.semilogx()
    plt.legend(frameon=False, ncol=2, fontsize="small", title="p+p")
    plt.xlabel("$E_\\mathrm{cm}$ / GeV");

KeyError: 'Pythia8'

<Figure size 640x480 with 0 Axes>